<a href="https://colab.research.google.com/github/venkat2020-github/DeepLearning/blob/main/OptimizeCNNModel_kerastuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Create CNN Model and Optimize it using Keras Tuner

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 133 kB 5.5 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.8.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
from platform import python_version
print(python_version())

3.7.13


In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
train_images=train_images/255.0

In [9]:
test_images=test_images/255.0

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
                                activation='relu',
                                input_shape=(28,28,1)),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
                                kernel_size=hp.Choice('conv_2_kernel',values = [3,5]),
                                activation='relu'
                                ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation='relu'
                                ),
                            keras.layers.Dense(10,activation='softmax')
                            ])
       
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [16]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [18]:
#tuner_search.search(train_images, train_labels, ephocs=3, validation_split=0.1)
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 48s]
val_accuracy: 0.8818333148956299

Best val_accuracy So Far: 0.9141666889190674
Total elapsed time: 00h 07m 14s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 80)        23120     
                                                                 
 flatten (Flatten)           (None, 46080)             0         
                                                                 
 dense (Dense)               (None, 48)                2211888   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 2,235,818
Trainable params: 2,235,818
Non-trainable params: 0
_________________________________________________________________


In [22]:
#model.fit(train_images, train_labels, ephocs=10,validation_split=0.1,initial_epoch=3)
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1417 - accuracy: 0.9486 - val_loss: 0.2360 - val_accuracy: 0.9173
Epoch 5/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1026 - accuracy: 0.9634 - val_loss: 0.2612 - val_accuracy: 0.9148
Epoch 6/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0733 - accuracy: 0.9728 - val_loss: 0.3039 - val_accuracy: 0.9142
Epoch 7/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0528 - accuracy: 0.9812 - val_loss: 0.3526 - val_accuracy: 0.9172
Epoch 8/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0392 - accuracy: 0.9854 - val_loss: 0.3921 - val_accuracy: 0.9143
Epoch 9/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0319 - accuracy: 0.9883 - val_loss: 0.4240 - val_accuracy: 0.9150
Epoch 10/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0257 - accuracy: 0.9911 - val_loss: 0.4942 -